# First step of the hematopoiesis application.

---------

## Table of content

[**Data preprocessing**](#datapreprocess)
1. [Import the observations built from single-cell data.](#import)
2. [Standardize the observations' gene names.](#standardobs)
3. [Get interaction graph from DoRothEA database.](#dorothea)

[**Use of BoNesis for selection of components**](#bonesis)

---------

## Data preprocessing <a class="anchor" id="datapreprocess"></a>

### 1. Import the observations built from single-cell data: <a class="anchor" id="import"></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_nesto = pd.read_csv("data/nestorowa_binarizedObservations.csv", sep=",", index_col=[0])

In [3]:
df_nesto

,1110008L16Rik,1110059E24Rik,1200007C13Rik,1300017J02Rik,1500005C15Rik,1600014C10Rik,1600020E01Rik,1700006J14Rik,1700017B05Rik,1700024P16Rik,...,Zscan21,Zscan22,Zscan29,Zswim3,Zswim4,Zufsp,Zxdb,Zxdc,Zyx,Zzz3
S1,0,0,NaN,NaN,NaN,0,0,0,0,NaN,...,0,0,0,NaN,NaN,0,NaN,NaN,1,0
S4,0,1,NaN,NaN,NaN,1,0,0,0,NaN,...,0,0,0,NaN,NaN,0,NaN,NaN,1,0
S0,0,1,NaN,NaN,NaN,0,0,0,0,NaN,...,0,0,0,NaN,NaN,0,NaN,NaN,1,0
S2,0,0,NaN,NaN,NaN,1,1,0,0,NaN,...,0,0,0,NaN,NaN,0,NaN,NaN,1,0
S5,0,0,NaN,NaN,NaN,0,0,1,0,NaN,...,0,0,0,NaN,NaN,0,NaN,NaN,1,0
S3,0,0,NaN,NaN,NaN,0,0,0,1,NaN,...,0,0,0,NaN,NaN,0,NaN,NaN,1,1


In [4]:
observations_from_singlecell_nestorowa = df_nesto.to_dict(orient="index")

In [ ]:
observations_from_singlecell_nestorowa

### 2. Standardize the observations' gene names. <a class="anchor" id="standardobs"></a>

We use the file "data/Mus_musculus.gene_info.202103.tsv" downloaded on 2021-03 from NCBI (mouse gene info).

In [5]:
import gene_name_standardization as gns

In [6]:
standardized_observations = gns.observations_standardization(observations_from_singlecell_nestorowa, "data/Mus_musculus.gene_info.202103.tsv")

In [7]:
# Visualisation of the matrix with standardized gene names:
df_standardized = pd.DataFrame.from_dict(standardized_observations, orient="index").fillna('')
df_standardized

,PRORP,1110059E24RIK,1200007C13RIK,1300017J02RIK,1500005C15RIK,1600014C10RIK,1600020E01RIK,1700006J14RIK,1700017B05RIK,FYB2,...,ZSCAN21,ZSCAN22,ZSCAN29,ZSWIM3,ZSWIM4,ZUP1,ZXDB,ZXDC,ZYX,ZZZ3
S1,0,0,,,,0,0,0,0,,...,0,0,0,,,0,,,1,0
S4,0,1,,,,1,0,0,0,,...,0,0,0,,,0,,,1,0
S0,0,1,,,,0,0,0,0,,...,0,0,0,,,0,,,1,0
S2,0,0,,,,1,1,0,0,,...,0,0,0,,,0,,,1,0
S5,0,0,,,,0,0,1,0,,...,0,0,0,,,0,,,1,0
S3,0,0,,,,0,0,0,1,,...,0,0,0,,,0,,,1,1


In [8]:
# Genes being in this single-cell dataset:
standardized_genenames_in_singlecell = set(df_standardized)

In [9]:
len(standardized_genenames_in_singlecell)

4768

### 3. Get interaction graph from DoRothEA database. <a class="anchor" id="dorothea"></a>

Interactions of confidence A, B and C were extracted from DoRothEA on 2021-07-07. 
The extraction from DoRothEA and the standardization of the gene names were processed as described in the [tutorial](https://github.com/StephanieChevalier/notebooks_for_bonesis/blob/main/tutorials/Tutorial_for_interaction_graph_preprocessing.ipynb). 
We only kept edges from transcription factors to transcription factors or to genes with expression values in the Nestorowa single-cell dataset ([dorothea_ABC_20210707_TF.sif](https://github.com/StephanieChevalier/notebooks_for_bonesis/blob/main/applications/hematopoiesis/data/dorothea_ABC_20210707_TF.sif): 1001 nodes, 2777 edges).

## Use of BoNesis for selection of components <a class="anchor" id="bonesis"></a>

In [10]:
import bonesis

In [11]:
standardized_dorothea = bonesis.InfluenceGraph.from_sif("data/dorothea_ABC_20210707_TF.sif", maxclause=8, allow_skipping_nodes=True, canonic=False)

In [12]:
bo = bonesis.BoNesis(standardized_dorothea, standardized_observations)

In [13]:
print(f"domain: {len(standardized_dorothea.nodes())} nodes, {len(standardized_dorothea.edges())} edges")

domain: 1001 nodes, 2790 edges


2790 edges vs 2777 into the SIF file: the difference comes from the add of a positive loop on each node without input (indegree 0).

### Dynamics
<img src="img/trajectoire.png" alt="nestorowa stream trajectory" style="width:40%;"/>

#### fixpoints

In [14]:
s2 = bo.fixed(~bo.obs("S2"))
s4 = bo.fixed(~bo.obs("S4"))
s5 = bo.fixed(~bo.obs("S5"))
s2 != s4
s5 != s4
s2 != s5;

#### positive reachability

In [15]:
~bo.obs('S1') >= ~bo.obs('S0') >= s2
~bo.obs('S0') >= ~bo.obs('S3') >= s4
~bo.obs('S3') >= s5;

#### negative reachability

In [16]:
~bo.obs("S3") / s2;

### Optimization & view

In [17]:
bo.maximize_nodes()
bo.maximize_strong_constants()

<bonesis.language.ManagedIface.__init__.<locals>.managed.<locals>.Managed at 0x7fa23974eac0>

In [18]:
view = bonesis.NonStrongConstantNodesView(bo, mode="opt")

In [ ]:
view.standalone(output_filename=f"maxnodes_maxstrongconstant.sh")